In [1]:
import os

# Set your desired cache directory
os.environ['HF_HOME'] = 'D:\Repositories\multilingual_mice\.cache'

In [2]:
import pandas as pd
import sys
sys.path.append("..")
from mmice.utils import html_highlight_diffs
from mmice.edit_finder import EditEvaluator
from mmice.maskers.random_masker import RandomMasker
from transformers import MT5TokenizerFast
from IPython.display import display, HTML
import numpy as np
import spacy
from tqdm import tqdm

nlp = spacy.load("en_core_web_sm")

eval = EditEvaluator(fluency_model_name="google/mt5-small",
                     fluency_masker=RandomMasker(None, MT5TokenizerFast.from_pretrained("google/mt5-small",
                                                                                        # force_download=True,
                                                                                        model_max_length=700, legacy=False), 700))

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5TokenizerFast'.
d:\Repositories\multilingual_mice\.venv\lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
FIX_FLAG = False
LOAD_BEST = True
TASK = "42k_hcuch"
STAGE2EXP = "mmice-mt5-small-lora-00"
SAVE_PATH = f"../results/{TASK}/edits/{STAGE2EXP}/"
EDIT_PATH = SAVE_PATH + "edits.csv"

In [4]:
def read_edits(path):
    edits = pd.read_csv(path, sep="\t", lineterminator="\n").dropna()
    edits = edits[edits['data_idx'] != 'data_idx']
    if edits['new_pred'].dtype == np.dtype('float64'):
        edits['new_pred'] = edits.apply(lambda row: str(int(row['new_pred']) if not np.isnan(row['new_pred']) else ""), axis=1)
        edits['orig_pred'] = edits.apply(lambda row: str(int(row['orig_pred']) if not np.isnan(row['orig_pred']) else ""), axis=1)
        edits['contrast_pred'] = edits.apply(lambda row: str(int(row['contrast_pred']) if not np.isnan(row['contrast_pred']) else ""), axis=1)
    else:
        edits['new_pred'].fillna(value="", inplace=True)
        edits['orig_pred'].fillna(value="", inplace=True)
        edits['contrast_pred'].fillna(value="", inplace=True)
    return edits

In [5]:
def get_best_edits(edits):
    """ MiCE writes all edits that are found in Stage 2, 
    but we only want to evaluate the smallest per input. 
    Calling get_sorted_e() """
    edits['sorted_idx'] = pd.to_numeric(edits['sorted_idx'])
    edits['minimality'] = pd.to_numeric(edits['minimality'])
    edits['data_idx'] = pd.to_numeric(edits['data_idx'])
    edits['duration'] = pd.to_numeric(edits['duration'])
    return edits[edits['sorted_idx'] == 0]
    
def evaluate_edits(edits):
    temp = edits[edits['sorted_idx'] == 0]
    minim = temp['minimality'].mean()
    flipped = temp[temp['new_pred'].astype(str)==temp['contrast_pred'].astype(str)]
    nunique = temp['data_idx'].nunique()
    
    flip_rate = len(flipped)/nunique
    duration = temp['duration'].mean()
    metrics = {
        "num_total": nunique,
        "num_flipped": len(flipped),
        "flip_rate": flip_rate,
        "minimality": minim,
        #"fluency": temp['fluency'].mean(),
        "duration": duration,
    }
    for k, v in metrics.items():
        print(f"{k}: \t{round(v, 3)}")
    return metrics

In [6]:
def display_edits(row):
    html_original, html_edited = html_highlight_diffs(row['orig_editable_seg'], row['edited_editable_seg'], nlp)
    minim = round(row['minimality'], 3)
    print(f"MINIMALITY: \t{minim}")
    print("")
    display(HTML(html_original))
    display(HTML(html_edited))

def display_classif_results(rows):
    for _, row in rows.iterrows():
        orig_contrast_prob_pred = round(row['orig_contrast_prob_pred'], 3)
        new_contrast_prob_pred = round(row['new_contrast_prob_pred'], 3)
        print("-----------------------")
        print(f"ORIG LABEL: \t{row['orig_pred']}")
        print(f"CONTR LABEL: \t{row['contrast_pred']} (Orig Pred Prob: {orig_contrast_prob_pred})")
        print(f"NEW LABEL: \t{row['new_pred']} (New Pred Prob: {new_contrast_prob_pred})")
        print("")
        display_edits(row)

def display_race_results(rows):
    for _, row in rows.iterrows():
        orig_contrast_prob_pred = round(row['orig_contrast_prob_pred'], 3)
        new_contrast_prob_pred = round(row['new_contrast_prob_pred'], 3)
        orig_input = eval(row['orig_input'])
        options = orig_input['options']
        print("-----------------------")
        print(f"QUESTION: {orig_input['question']}")
        print("\nOPTIONS:")
        for opt_idx, opt in enumerate(options):
            print(f"  ({opt_idx}) {opt}")
        print(f"\nORIG LABEL: \t{row['orig_pred']}")
        print(f"CONTR LABEL: \t{row['contrast_pred']} (Orig Pred Prob: {orig_contrast_prob_pred})")
        print(f"NEW LABEL: \t{row['new_pred']} (New Pred Prob: {new_contrast_prob_pred})")
        print("")
        display_edits(row)

In [7]:
def fix_windows_corruption(file_path):
    new_file_path = file_path.split("/")
    new_file_path[-1] = 'fixed_edits.csv'
    new_file_path = "/".join(new_file_path)
    with open(file_path, 'r') as f, open(new_file_path, 'w+') as new_file:
        i = 0
        for line in f:
            i += 1
            line_ = line.replace(";", "").strip()
            if len(line_) == 0:
                continue
            if line_[0] == "\"":
                line_ = line_[1:]
            if line_[-1] == "\"":
                line_ = line_[:-1] 
            new_file.write(line_ + "\n")
    return new_file_path


if FIX_FLAG:
    EDIT_PATH = fix_windows_corruption(EDIT_PATH)           

In [8]:
edits = read_edits(EDIT_PATH) #if not LOAD_BEST else read_edits(SAVE_PATH + "best_edits.csv")
edits = get_best_edits(edits)

C:\Users\domin\AppData\Local\Temp\ipykernel_1256\538214286.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  edits['new_pred'].fillna(value="", inplace=True)
C:\Users\domin\AppData\Local\Temp\ipykernel_1256\538214286.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

In [9]:
edits.head()

,data_idx,sorted_idx,orig_pred,new_pred,contrast_pred,orig_contrast_prob_pred,new_contrast_prob_pred,orig_input,edited_input,orig_editable_seg,edited_editable_seg,minimality,num_edit_rounds,mask_frac,duration,error\r\r
1,6,0,nodulos,nodulos,NO nodulos,0.009815,0.999944,Árbol traqueo-bronquial principal permeable. P...,Árbol traqueo-bronquial principal permeable. P...,Árbol traqueo-bronquial principal permeable. P...,Árbol traqueo-bronquial principal permeable. P...,0.039589,1.0,0.034375,24.195103,False\r\r
51,4,0,nodulos,nodulos,NO nodulos,0.048486,0.955424,Árbol tráqueo-bronquial principal permeable. G...,Árbol tráqueo-bronquial principal permeable. G...,Árbol tráqueo-bronquial principal permeable. G...,Árbol tráqueo-bronquial principal permeable. G...,0.037862,1.0,0.034375,17.145000,False\r\r


In [ ]:
from evaluate import load
perplexity = load("perplexity", module_type="metric")

edited_results = perplexity.compute(predictions=edits['edited_input'].tolist()[:], model_id='facebook/xglm-1.7B', batch_size=1)
orig_results = perplexity.compute(predictions=edits['orig_input'].tolist()[:], model_id='facebook/xglm-1.7B', batch_size=1)

In [ ]:
print(edited_results['mean_perplexity'])
print(orig_results['mean_perplexity'])


edits['edit_perplexity'] = edited_results['perplexities']
edits['orig_perplexity'] = orig_results['perplexities']
edits.to_parquet(SAVE_PATH + "best_edits.parquet.gzip",
                 compression='gzip')

In [ ]:
# tqdm.pandas(desc='original sequence loss!')
# a = edits["orig_editable_seg"].progress_apply(lambda x: eval.score_fluency(x, 2))

# tqdm.pandas(desc='edited sequence loss!')
# b = edits["edited_editable_seg"].progress_apply(lambda x: eval.score_fluency(x) if isinstance(x, str) else 0)

# edits['fluency'] =  b/a
# edits.to_csv(SAVE_PATH + "best_edits.csv", sep="\t", lineterminator="\n")

In [10]:
#edits = read_edits(SAVE_PATH + "best_edits.csv")
#edits = get_best_edits(edits)
metrics = evaluate_edits(edits)

num_total: 	2
num_flipped: 	0
flip_rate: 	0.0
minimality: 	0.039
duration: 	20.67


In [11]:
random_rows = edits.sample(1)
display_classif_results(random_rows)
# display_race_results(random_rows)

-----------------------
ORIG LABEL: 	nodulos
CONTR LABEL: 	NO nodulos (Orig Pred Prob: 0.01)
NEW LABEL: 	nodulos (New Pred Prob: 1.0)

MINIMALITY: 	0.04

